In [58]:
import pandas as pd
import string
from collections import *
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import collections
import numpy as np
import sys  
import re
from textblob import Blobber
from textblob.en.sentiments import NaiveBayesAnalyzer
import graphlab as gl
reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
data = gl.SFrame.read_csv('Hygiene/hygiene.dat',header=False,delimiter='\n\n').rename({'X1':'Review'})

Finished parsing file C:\Users\thienbui\Documents\Data\data_mining_c\Hygiene\hygiene.dat

Parsing completed. Parsed 100 lines in 0.943766 secs.

Read 6770 lines. Lines per second: 6980.22

Finished parsing file C:\Users\thienbui\Documents\Data\data_mining_c\Hygiene\hygiene.dat

Parsing completed. Parsed 13299 lines in 1.26463 secs.

In [3]:
data['label'] = gl.SFrame.read_csv('Hygiene/hygiene.dat.labels',header=None,delimiter='\n',column_type_hints=str)['X1']

Finished parsing file C:\Users\thienbui\Documents\Data\data_mining_c\Hygiene\hygiene.dat.labels

Parsing completed. Parsed 13299 lines in 0.016082 secs.

In [4]:
data.add_columns(gl.SFrame.read_csv('Hygiene/hygiene.dat.additional',
                   header=None).rename({'X1':'Cate','X2':'zipCode','X3':'nreview','X4':'avg_star'}))

Finished parsing file C:\Users\thienbui\Documents\Data\data_mining_c\Hygiene\hygiene.dat.additional

Parsing completed. Parsed 100 lines in 0.062347 secs.

Finished parsing file C:\Users\thienbui\Documents\Data\data_mining_c\Hygiene\hygiene.dat.additional

Parsing completed. Parsed 13299 lines in 0.033589 secs.

Review,label,Cate,zipCode,nreview,avg_star
"The baguettes and rollsare excellent, and ...",1,"[Vietnamese, Sandwiches,Restaurants] ...",98118,4,4.0
I live up the street fromBetty. &#160;When my ...,1,"[American (New),Restaurants] ...",98109,21,4.04761904762
I'm worried about how Iwill review this place ...,1,"[Mexican, Restaurants]",98103,14,3.11111111111
Why can't you access themon Google street view? ...,0,"[Mexican, Tex-Mex,Restaurants] ...",98112,42,4.08888888889
Things to like about thisplace: homemade ...,0,"[Mexican, Restaurants]",98102,12,3.07142857143
I had been holding off onvisiting Bastille for ...,1,"[Breakfast & Brunch,French, Restaurants] ...",98107,59,3.63492063492
I had gone by this placeas they were moving in ...,1,"[Creperies, Restaurants]",98105,6,4.16666666667
Any chance I get to eatwith my hands and hav ...,0,"[Vegetarian, Ethiopian,Vegan, Restaurants] ...",98102,17,3.72222222222
My favorite Thairestaurant in the ...,0,"[Thai, Restaurants]",98105,16,3.58823529412
I'm pretty sure someonewho was born and raised ...,0,"[Barbeque, Restaurants]",98108,19,3.09523809524


In [5]:
data['Cate'] = data['Cate'].apply(lambda x: dict(Counter(x)))
data = data.unpack('Cate')

In [6]:
data['Review'] = data['Review'].apply(lambda x: re.sub('[^a-z]+', ' ', x.lower()))

In [7]:
#data['tf_idf'] = gl.text_analytics.tf_idf(gl.text_analytics.count_words(data['Review']))
data['tf_idf'] = gl.text_analytics.tf_idf(gl.text_analytics.count_ngrams(data['Review']))

In [8]:
m = gl.topic_model.create(data['tf_idf'],method='alias',num_iterations=20)

Learning a topic model

Number of documents     13299

Vocabulary size   2085780

Initializing topic assignments

Constructed alias samplers:17.5685s

...

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity | Elapsed for perp. |

+-----------+---------------+----------------+-----------------+-------------------+

| 10        | 4m 15s        | 2.70973e+006   | 0               | 0                 |

| 20        | 7m 56s        | 3.51134e+006   | 0               | 0                 |

+-----------+---------------+----------------+-----------------+-------------------+

In [9]:
data['prob'] = m.predict(data['tf_idf'],output_type='probability')

In [10]:
def list2dict(l):
    rep = dict()
    for i in range(10):
        rep[i] = l[i]
    return rep
data['prob'] = data['prob'].apply(list2dict)

In [11]:
data.remove_columns(['tf_idf','Review'])

label,zipCode,nreview,avg_star,Cate.Afghan,Cate.African,Cate.American (New),Cate.American(Traditional) ...
1,98118,4,4.0,None,None,None,None
1,98109,21,4.04761904762,None,None,1,None
1,98103,14,3.11111111111,None,None,None,None
0,98112,42,4.08888888889,None,None,None,None
0,98102,12,3.07142857143,None,None,None,None
1,98107,59,3.63492063492,None,None,None,None
1,98105,6,4.16666666667,None,None,None,None
0,98102,17,3.72222222222,None,None,None,None
0,98105,16,3.58823529412,None,None,None,None
0,98108,19,3.09523809524,None,None,None,None


In [12]:
data = data.unpack('prob')

In [13]:
features = data.column_names()
for col in features:
    data = data.fillna(column=col,value=0)

In [14]:
leftbh = data[data['label']=='[None]']

In [15]:
aval = data[data['label']!='[None]']

In [16]:
aval['label'] = aval['label'].apply(int)

In [17]:
train,test = aval.random_split(fraction=.9,seed=317)

In [18]:
train,valid = train.random_split(fraction=.8,seed=317)
m = gl.random_forest_classifier.create(train, target='label',random_seed=317,validation_set=valid)

WARNING: Detected extremely low variance for feature(s) 'Cate.Afghan', 'Cate.African', 'Cate.Australian', 'Cate.Belgian', 'Cate.Brazilian', 'Cate.British', 'Cate.Cambodian', 'Cate.Chicken Wings', 'Cate.Colombian', 'Cate.Comfort Food', 'Cate.Cuban', 'Cate.Egyptian', 'Cate.Filipino', 'Cate.Fish & Chips', 'Cate.Fondue', 'Cate.Food Court', 'Cate.Gastropubs', 'Cate.German', 'Cate.Indonesian', 'Cate.Kosher', 'Cate.Laotian', 'Cate.Lebanese', 'Cate.Malaysian', 'Cate.Mongolian', 'Cate.Persian/Iranian', 'Cate.Polish', 'Cate.Puerto Rican', 'Cate.Restaurants', 'Cate.Scandinavian', 'Cate.Scottish', 'Cate.Senegalese', 'Cate.Shanghainese', 'Cate.Trinidadian', 'Cate.Turkish', 'Cate.Venezuelan' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Random forest classifier:

--------------------------------------------------------

Number of examples          : 400

Number of classes           : 2

...

| 3         | 0.018034     | 0.845000          | 0.577982            | 0.463319          | 0.735031            |

| 4         | 0.019538     | 0.860000          | 0.559633            | 0.460365          | 0.727569            |

| 5         | 0.021042     | 0.875000          | 0.568807            | 0.446494          | 0.719409            |

| 6         | 0.022546     | 0.867500          | 0.550459            | 0.452656          | 0.717280            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [19]:
gl.evaluation.f1_score(predictions=m.predict(test),targets=test['label'])

0.7441860465116279

In [20]:
m = gl.boosted_trees_classifier.create(train, target='label',random_seed=317,validation_set=valid)

WARNING: Detected extremely low variance for feature(s) 'Cate.Afghan', 'Cate.African', 'Cate.Australian', 'Cate.Belgian', 'Cate.Brazilian', 'Cate.British', 'Cate.Cambodian', 'Cate.Chicken Wings', 'Cate.Colombian', 'Cate.Comfort Food', 'Cate.Cuban', 'Cate.Egyptian', 'Cate.Filipino', 'Cate.Fish & Chips', 'Cate.Fondue', 'Cate.Food Court', 'Cate.Gastropubs', 'Cate.German', 'Cate.Indonesian', 'Cate.Kosher', 'Cate.Laotian', 'Cate.Lebanese', 'Cate.Malaysian', 'Cate.Mongolian', 'Cate.Persian/Iranian', 'Cate.Polish', 'Cate.Puerto Rican', 'Cate.Restaurants', 'Cate.Scandinavian', 'Cate.Scottish', 'Cate.Senegalese', 'Cate.Shanghainese', 'Cate.Trinidadian', 'Cate.Turkish', 'Cate.Venezuelan' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 400

Number of classes           : 2

...

| 3         | 0.008522     | 0.905000          | 0.541284            | 0.465018          | 0.705930            |

| 4         | 0.010527     | 0.905000          | 0.568807            | 0.429568          | 0.733936            |

| 5         | 0.011530     | 0.907500          | 0.550459            | 0.403339          | 0.750401            |

| 6         | 0.013035     | 0.917500          | 0.532110            | 0.375698          | 0.770972            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [21]:
gl.evaluation.f1_score(predictions=m.predict(test),targets=test['label'])

0.6829268292682926

In [56]:
m = gl.nearest_neighbor_classifier.create(train, target='label',distance='squared_euclidean')

Starting brute force nearest neighbors model training.

In [57]:
gl.evaluation.f1_score(predictions=m.predict(test),targets=test['label'])

Starting blockwise querying.

max rows per data block: 8080

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

+--------------+---------+-------------+--------------+

| 37           | 1850    | 12.5        | 1.501ms      |

| Done         | 14800   | 100         | 3.005ms      |

+--------------+---------+-------------+--------------+

0.7027027027027026

In [31]:
m = gl.svm_classifier.create(train, target='label',validation_set=valid,max_iterations=10000)

WARNING: Detected extremely low variance for feature(s) 'Cate.Afghan', 'Cate.African', 'Cate.Australian', 'Cate.Belgian', 'Cate.Brazilian', 'Cate.British', 'Cate.Cambodian', 'Cate.Chicken Wings', 'Cate.Colombian', 'Cate.Comfort Food', 'Cate.Cuban', 'Cate.Egyptian', 'Cate.Filipino', 'Cate.Fish & Chips', 'Cate.Fondue', 'Cate.Food Court', 'Cate.Gastropubs', 'Cate.German', 'Cate.Indonesian', 'Cate.Kosher', 'Cate.Laotian', 'Cate.Lebanese', 'Cate.Malaysian', 'Cate.Mongolian', 'Cate.Persian/Iranian', 'Cate.Polish', 'Cate.Puerto Rican', 'Cate.Restaurants', 'Cate.Scandinavian', 'Cate.Scottish', 'Cate.Senegalese', 'Cate.Shanghainese', 'Cate.Trinidadian', 'Cate.Turkish', 'Cate.Venezuelan' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

SVM:

--------------------------------------------------------

Number of examples          : 400

Number of classes           : 2

...

| 100       | 147      | 0.500000  | 1.517025     | 0.740000          | 0.577982            |

| 101       | 148      | 0.500000  | 1.528556     | 0.740000          | 0.577982            |

| 200       | 296      | 1.000000  | 2.906220     | 0.740000          | 0.577982            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [32]:
gl.evaluation.f1_score(predictions=m.predict(test),targets=test['label'])

0.6842105263157895

In [44]:
m =gl.logistic_classifier.create(dataset=train,target='label',validation_set=valid,l1_penalty=.1,l2_penalty=.1,max_iterations=100)

WARNING: Detected extremely low variance for feature(s) 'Cate.Afghan', 'Cate.African', 'Cate.Australian', 'Cate.Belgian', 'Cate.Brazilian', 'Cate.British', 'Cate.Cambodian', 'Cate.Chicken Wings', 'Cate.Colombian', 'Cate.Comfort Food', 'Cate.Cuban', 'Cate.Egyptian', 'Cate.Filipino', 'Cate.Fish & Chips', 'Cate.Fondue', 'Cate.Food Court', 'Cate.Gastropubs', 'Cate.German', 'Cate.Indonesian', 'Cate.Kosher', 'Cate.Laotian', 'Cate.Lebanese', 'Cate.Malaysian', 'Cate.Mongolian', 'Cate.Persian/Iranian', 'Cate.Polish', 'Cate.Puerto Rican', 'Cate.Restaurants', 'Cate.Scandinavian', 'Cate.Scottish', 'Cate.Senegalese', 'Cate.Shanghainese', 'Cate.Trinidadian', 'Cate.Turkish', 'Cate.Venezuelan' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

Number of examples          : 400

Number of classes           : 2

...

| 5         | 6        | 0.002284  | 0.196082     | 0.727500          | 0.541284            |

| 6         | 7        | 0.002284  | 0.217132     | 0.725000          | 0.541284            |

| 11        | 12       | 0.002284  | 0.302359     | 0.727500          | 0.559633            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [46]:
gl.evaluation.f1_score(predictions=m.predict(test),targets=test['label'])

0.7692307692307692